In [1]:
import numpy as np
import pandas as pd
import re
import cv2

In [2]:
!pwd

/media/paolo/Volume/matteo/unimi/tesi_master/code/fear_gen


In [3]:
import os

folder_path = "./data/gaze/ROI" # sostituire "percorso_della_cartella" con il percorso della cartella che si vuole leggere

all_roi = []
for filename in os.listdir(folder_path):
    path_complete = os.path.join(folder_path, filename)
    if os.path.isfile(path_complete):
        all_roi.append(path_complete)

all_roi




['./data/gaze/ROI/roiLook018.asc.npy',
 './data/gaze/ROI/roiLook001.asc.npy',
 './data/gaze/ROI/roiLook002.asc.npy',
 './data/gaze/ROI/roiLook003.asc.npy',
 './data/gaze/ROI/roiLook004.asc.npy',
 './data/gaze/ROI/roiLook005.asc.npy',
 './data/gaze/ROI/roiLook006.asc.npy',
 './data/gaze/ROI/roiLook007.asc.npy',
 './data/gaze/ROI/roiLook008.asc.npy',
 './data/gaze/ROI/roiLook010.asc.npy',
 './data/gaze/ROI/roiLook011.asc.npy',
 './data/gaze/ROI/roiLook012.asc.npy',
 './data/gaze/ROI/roiLook013.asc.npy',
 './data/gaze/ROI/roiLook014.asc.npy',
 './data/gaze/ROI/roiLook015.asc.npy',
 './data/gaze/ROI/roiLook016.asc.npy',
 './data/gaze/ROI/roiLook017.asc.npy',
 './data/gaze/ROI/roiLook019.asc.npy',
 './data/gaze/ROI/roiLook020.asc.npy',
 './data/gaze/ROI/roiLook021.asc.npy',
 './data/gaze/ROI/roiLook022.asc.npy',
 './data/gaze/ROI/roiLook023.asc.npy',
 './data/gaze/ROI/roiLook024.asc.npy',
 './data/gaze/ROI/roiLook025.asc.npy',
 './data/gaze/ROI/roiLook026.asc.npy',
 './data/gaze/ROI/roiLook

In [4]:
def extract_sub(path):
    pattern = r"roiLook(\d+)\.asc\.npy"
    return int(re.search(pattern, path).group(1))

In [4]:
list_subject = [1]

In [ ]:
df_ = pd.DataFrame(columns=['Subject', 'Trial', 'Fixation #', 'X_Coord', 'Y_Coord'])

for path in all_roi:
    data = np.load(path, allow_pickle=True)
    for i in range(len(data)):
        subject = extract_sub(path)
        trial = i+1
        fix_ = data[i]['fixations']
        for num, xy in enumerate(fix_):
            coords = tuple(np.round(xy['mean'], 2))
            row = [int(subject), int(trial), int(num+1), coords[0], coords[1]]
            df_.loc[len(df_)] = row

In [ ]:
df_

# convert vincenzo npy of feature into dataframe

In [ ]:
df_ = pd.DataFrame(columns=['Subject', 'Trial', 'Fixation #', 'Feature'])

for path in all_roi:
    data = np.load(path, allow_pickle=True)
    for i in range(len(data)):
        subject = extract_sub(path)
        trial = i+1
        fix_ = data[i]['fixations']
        for num, xy in enumerate(fix_):
            coords = tuple(np.round(xy['mean'], 2))
            row = [int(subject), int(trial), int(num+1), coords[0], coords[1]]
            df_.loc[len(df_)] = row

In [ ]:
len(df_)

In [ ]:
df_.to_csv('../data/gaze/tmp/tmp_fixations.csv', index=False)

In [ ]:
baseline = pd.read_csv('./data/gaze/baselines/18.csv')
baseline

In [43]:
df_updated = df_.copy(deep=True)
x_cross = 1920/2
y_cross = 1080/2

In [ ]:
all_roi

In [ ]:
fixations = pd.read_csv('./data/gaze/tmp/tmp_fixations.csv')


# for all subjects
for path in all_roi:
    subj = extract_sub(path)
    print("subject: "+str(subj))
    baseline = pd.read_csv('./data/gaze/baselines/'+str(subj)+'.csv')
    print(baseline)
    sub_fixations = fixations[fixations['Subject'] == subj]
    print(sub_fixations)
    break

In [ ]:
baseline

In [ ]:
fixations_updated = fixations.copy(deep=True)
fixations_updated

In [ ]:
df_

# baseline correction, insert missing trials

In [70]:
# for all subjects
for path in all_roi:
    subject = extract_sub(path)
    baseline = pd.read_csv('./data/gaze/baselines/'+str(subject)+'.csv')
    for i in range (1,161):
        current_trial_exist = (len(baseline[baseline['TRIAL_LABEL']==i])!=0)
        if not current_trial_exist:
            baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
    baseline.to_csv('./data/gaze/corrected_baselines/'+str(subject)+'.csv',index=False)

/tmp/ipykernel_118518/4175139398.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
/tmp/ipykernel_118518/4175139398.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
/tmp/ipykernel_118518/4175139398.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
/tmp/ipykernel_1185

# recalculate fixation using baseline

In [71]:
subject=18

In [72]:
baseline = pd.read_csv('./data/gaze/corrected_baselines/'+str(subject)+'.csv')

In [73]:
baseline

,RECORDING_SESSION_LABEL,TRIAL_LABEL,CURRENT_FIX_X,CURRENT_FIX_Y,IS_VALID
0,18,1,977.232099,448.003704,True
1,18,2,938.900000,454.100000,True
2,18,3,875.600000,542.300000,True
3,18,4,938.512274,447.137184,True
4,18,5,930.966917,447.110902,True
...,...,...,...,...,...
155,18,63,-1.000000,-1.000000,False
156,18,65,-1.000000,-1.000000,False
157,18,73,-1.000000,-1.000000,False
158,18,113,-1.000000,-1.000000,False


In [84]:
fixations = pd.read_csv('./data/gaze/tmp/tmp_fixations.csv')
fixations

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,52.0,1.0,1.0,942.04,528.50
1,52.0,1.0,2.0,946.56,520.01
2,52.0,1.0,3.0,1005.51,482.76
3,52.0,1.0,4.0,1061.42,473.28
4,52.0,1.0,5.0,1057.73,474.32
...,...,...,...,...,...
222859,25.0,138.0,28.0,913.57,554.72
222860,25.0,138.0,29.0,919.40,553.31
222861,25.0,138.0,30.0,1082.11,557.33
222862,25.0,138.0,31.0,1075.46,536.27


In [85]:
fixations_updated = fixations.copy(deep=True)

In [86]:
# for all subjects
for path in all_roi:
    subject = extract_sub(path)
    baseline = pd.read_csv('./data/gaze/baselines/'+str(subject)+'.csv')

    for i in range(len(baseline)):
        trial = baseline.TRIAL_LABEL[i]
        valid = baseline.IS_VALID[i]

        if valid:
            x_base = baseline.CURRENT_FIX_X[i]
            y_base = baseline.CURRENT_FIX_Y[i]

        else:
            x_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_X'].mean()
            y_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_Y'].mean()

        delta_x, delta_y = (x_cross-x_base), (y_cross-y_base)

        for row in fixations[fixations['Trial'] == trial].iterrows():
            fixations_updated.loc[row[0], 'X_Coord'] = np.round(row[1]['X_Coord'] + delta_x,2)
            fixations_updated.loc[row[0], 'Y_Coord'] = np.round(row[1]['Y_Coord'] + delta_y, 2)


In [89]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,52.0,1.0,1.0,991.74,512.40
1,52.0,1.0,2.0,996.26,503.91
2,52.0,1.0,3.0,1055.21,466.66
3,52.0,1.0,4.0,1111.12,457.18
4,52.0,1.0,5.0,1107.43,458.22
...,...,...,...,...,...
222859,25.0,138.0,28.0,968.47,524.52
222860,25.0,138.0,29.0,974.30,523.11
222861,25.0,138.0,30.0,1137.01,527.13
222862,25.0,138.0,31.0,1130.36,506.07


In [90]:
fixations_updated.to_csv('data/gaze/fixations.csv',index=False)

In [78]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,52.0,1.0,1.0,924.81,620.50
1,52.0,1.0,2.0,929.33,612.01
2,52.0,1.0,3.0,988.28,574.76
3,52.0,1.0,4.0,1044.19,565.28
4,52.0,1.0,5.0,1040.50,566.32
...,...,...,...,...,...
222859,25.0,138.0,28.0,933.97,577.42
222860,25.0,138.0,29.0,939.80,576.01
222861,25.0,138.0,30.0,1102.51,580.03
222862,25.0,138.0,31.0,1095.86,558.97


In [92]:
fixations_updated['ROI'] = np.nan

In [93]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord,ROI
0,52.0,1.0,1.0,991.74,512.40,NaN
1,52.0,1.0,2.0,996.26,503.91,NaN
2,52.0,1.0,3.0,1055.21,466.66,NaN
3,52.0,1.0,4.0,1111.12,457.18,NaN
4,52.0,1.0,5.0,1107.43,458.22,NaN
...,...,...,...,...,...,...
222859,25.0,138.0,28.0,968.47,524.52,NaN
222860,25.0,138.0,29.0,974.30,523.11,NaN
222861,25.0,138.0,30.0,1137.01,527.13,NaN
222862,25.0,138.0,31.0,1130.36,506.07,NaN


In [56]:
mask_12 = cv2.imread("data/gaze/mask/1&2 mask M010.png")
mask_34 = cv2.imread("data/gaze/mask/3&4 mask M057C.png")
mask_56 = cv2.imread("data/gaze/mask/5&6 mask F048C.png")
mask_78 = cv2.imread("data/gaze/mask/7&8 mask F101C.png")

libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile


In [57]:
roi_12 = set()
roi_34 = set()
roi_56 = set()
roi_78 = set()

for y,col in enumerate(mask_12):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_12.add((x+639,y+202))

for y,col in enumerate(mask_34):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_34.add((x+639,y+202))

for y,col in enumerate(mask_56):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_56.add((x+639,y+202))

for y,col in enumerate(mask_78):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_78.add((x+639,y+202))

# set of coordinates of eye region and mouth nose region

In [58]:
eye_roi = roi_12.union(roi_56)
mouth_nose_roi = roi_34.union(roi_78)

In [100]:
eye_roi

{(968, 540),
 (1042, 489),
 (1044, 535),
 (856, 484),
 (1021, 551),
 (1095, 500),
 (1056, 545),
 (868, 494),
 (835, 546),
 (899, 561),
 (1072, 489),
 (1049, 505),
 (1051, 551),
 (863, 500),
 (898, 494),
 (939, 495),
 (865, 546),
 (840, 489),
 (1015, 490),
 (1079, 505),
 (877, 556),
 (951, 505),
 (893, 500),
 (870, 516),
 (969, 495),
 (886, 460),
 (872, 562),
 (946, 511),
 (847, 505),
 (981, 505),
 (1022, 506),
 (923, 500),
 (1057, 500),
 (900, 516),
 (976, 511),
 (1036, 562),
 (953, 527),
 (1052, 506),
 (854, 521),
 (895, 522),
 (928, 470),
 (1029, 522),
 (930, 516),
 (1004, 465),
 (1045, 466),
 (907, 532),
 (1006, 511),
 (983, 527),
 (1082, 506),
 (999, 471),
 (1059, 522),
 (1034, 465),
 (1061, 568),
 (937, 532),
 (978, 533),
 (1013, 527),
 (1087, 476),
 (901, 471),
 (1066, 538),
 (1008, 533),
 (880, 533),
 (855, 476),
 (915, 527),
 (1020, 543),
 (832, 492),
 (834, 538),
 (908, 487),
 (873, 466),
 (1007, 466),
 (850, 482),
 (910, 533),
 (984, 482),
 (887, 549),
 (885, 476),
 (926, 477

In [101]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord,ROI
0,52.0,1.0,1.0,991.74,512.40,NaN
1,52.0,1.0,2.0,996.26,503.91,NaN
2,52.0,1.0,3.0,1055.21,466.66,NaN
3,52.0,1.0,4.0,1111.12,457.18,NaN
4,52.0,1.0,5.0,1107.43,458.22,NaN
...,...,...,...,...,...,...
222859,25.0,138.0,28.0,968.47,524.52,NaN
222860,25.0,138.0,29.0,974.30,523.11,NaN
222861,25.0,138.0,30.0,1137.01,527.13,NaN
222862,25.0,138.0,31.0,1130.36,506.07,NaN


In [119]:
for i,row in fixations_updated.iterrows():
    x = int(row['X_Coord'])
    y = int(row['Y_Coord'])
    if (x,y) in eye_roi:
        fixations_updated.loc[i, "ROI"] = "eye"
    if (x,y) in mouth_nose_roi:
        fixations_updated.loc[i, "ROI"] = "mouth_nose"

In [121]:
fixations_updated.to_csv('./data/gaze/fixations_with_ROI.csv',index=False)

In [3]:
fixations = pd.read_csv('./data/gaze/fixations_with_ROI.csv')
fixations

,Subject,Trial,Fixation #,X_Coord,Y_Coord,ROI
0,52.0,1.0,1.0,991.74,512.40,eye
1,52.0,1.0,2.0,996.26,503.91,eye
2,52.0,1.0,3.0,1055.21,466.66,NaN
3,52.0,1.0,4.0,1111.12,457.18,NaN
4,52.0,1.0,5.0,1107.43,458.22,NaN
...,...,...,...,...,...,...
222859,25.0,138.0,28.0,968.47,524.52,eye
222860,25.0,138.0,29.0,974.30,523.11,eye
222861,25.0,138.0,30.0,1137.01,527.13,NaN
222862,25.0,138.0,31.0,1130.36,506.07,NaN


In [4]:
subjects = fixations.Subject.unique()

In [49]:
for sub in subjects:
    sub = int(sub)
    fix_sub = fixations[fixations['Subject']==sub]
    fix_sub['feature'] = np.nan
    fix_sub['feature'] = fix_sub['feature'].astype('object')
    path = 'data/gaze/features_ou/event_features_'+str(sub)+".npy"
    feat_ou = np.load(path,allow_pickle=True)

    for trial in range(0,160):
        feat_trial = feat_ou[trial]['feat_fix'] #start from 0

        fix_sub_trial = fix_sub[fix_sub['Trial']==trial+1] #start from 1

        if not (len(feat_trial) == len(fix_sub_trial)):
            print(sub)
            continue
        #print('len\t',len(fix_sub_trial))

        i=0
        for index,fixation in fix_sub_trial.iterrows():
            fix_sub_trial.at[index, "feature"] = np.array(feat_trial[i])
            fix_sub[(fix_sub['Trial']==trial+1) & (fix_sub['Fixation #']==i+1)] = fix_sub_trial
            i+=1

        #print(fix_sub[(fix_sub['Trial']==trial+1)& (fix_sub['Fixation #']==i)])
        #print('-----------------------')

        #print(fix_sub)

    #here I have a df for each subject
    fix_sub.to_csv('data/gaze/fixations_feat_OU_and_ROI/fixations_features_OU_ROI_'+str(sub)+'.csv',index=False)


/tmp/ipykernel_232743/1505109967.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_sub['feature'] = np.nan
/tmp/ipykernel_232743/1505109967.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_sub['feature'] = fix_sub['feature'].astype('object')
/tmp/ipykernel_232743/1505109967.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

28


/tmp/ipykernel_232743/1505109967.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_sub['feature'] = np.nan
/tmp/ipykernel_232743/1505109967.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_sub['feature'] = fix_sub['feature'].astype('object')
/tmp/ipykernel_232743/1505109967.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

IndexError: index 138 is out of bounds for axis 0 with size 138

In [39]:
new = pd.read_csv('data/gaze/fixations_feat_OU_and_ROI/fixations_features_OU_ROI_52.csv')

In [43]:
def custom_parse_data(X):
  res_x =[]
  for i in range(len(X)):
    string = X[i]
    new_string = re.sub(r'\s+', ',', string)
    string_list = list(new_string)
    string_list[1] = ''
    new_string = ''.join(string_list)
    res_x.append(np.fromstring(new_string.strip('[]'), dtype=float, sep=','))

  return np.array(res_x)

In [46]:
custom_parse_data(new['feature'])

array([[ 5.58296898e-02, -8.16019419e-02,  1.55042773e-01, ...,
         2.64708645e+00,  7.78370192e+00,  1.85000000e+00],
       [ 1.44285533e-01, -2.17730798e-01,  2.06448540e+00, ...,
         8.98903823e+01,  1.73098582e+02,  7.00000000e-03],
       [ 3.02285988e-02, -4.31579589e-02,  1.73292581e-01, ...,
         8.69718610e+00,  1.15601009e+01,  1.75000000e-01],
       ...,
       [ 3.42518798e-02, -7.03157016e-02,  2.24850028e-01, ...,
         8.05453393e+00,  3.27170982e+01,  1.31000000e+00],
       [ 5.14828751e-02, -7.83152977e-02,  1.68982073e-01, ...,
         9.26446262e+00,  2.89681780e+01,  8.23000000e-01],
       [ 7.35594122e-02, -9.29941063e-02,  1.66701932e-01, ...,
         1.46657658e+01,  2.83208719e+01,  3.72000000e-01]])

In [47]:
new

,Subject,Trial,Fixation #,X_Coord,Y_Coord,ROI,feature
0,52.0,1.0,1.0,991.74,512.40,eye,[ 5.58296898e-02 -8.16019419e-02 1.55042773e-...
1,52.0,1.0,2.0,996.26,503.91,eye,[ 1.44285533e-01 -2.17730798e-01 2.06448540e+...
2,52.0,1.0,3.0,1055.21,466.66,NaN,[ 3.02285988e-02 -4.31579589e-02 1.73292581e-...
3,52.0,1.0,4.0,1111.12,457.18,NaN,[ 4.26937035e-02 -6.41844221e-02 1.48824097e-...
4,52.0,1.0,5.0,1107.43,458.22,NaN,[ 1.21799891e-01 -4.40269115e-02 8.24064904e-...
...,...,...,...,...,...,...,...
2806,52.0,160.0,11.0,1123.53,500.72,NaN,[ 2.14221844e-01 -1.74221518e-02 2.55173859e-...
2807,52.0,160.0,12.0,1087.11,453.75,NaN,[ 4.03370019e-02 -2.79626002e-02 1.37150341e-...
2808,52.0,160.0,13.0,1110.04,416.64,NaN,[ 3.42518798e-02 -7.03157016e-02 2.24850028e-...
2809,52.0,160.0,14.0,1105.09,410.36,NaN,[ 5.14828751e-02 -7.83152977e-02 1.68982073e-...


In [ ]:
#subj #trial #fix #xcoord ycoord region [feature]
group by subject,trial,region (mean su feature)

# extract saccades

In [33]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [37]:
df_ = pd.DataFrame(columns=['Subject', 'Trial', 'Saccades #', 'X_Start', 'Y_Start','X_End', 'Y_End'])

for path in all_roi:
    data = np.load(path, allow_pickle=True)
    subj = extract_sub(path)

    for trial in range(0,len(data)):
        sacc_list = data[trial]['saccades']

        index=0
        for event in (sacc_list):
            index+=1
            df_ = df_.append({'Subject':subj, 'Trial':trial+1, 'Saccades #':index,
                         'X_Start':event['start'][0], 'Y_Start':event['start'][1],
                         'X_End':event['end'][0], 'Y_End':event['end'][1] },
                       ignore_index=True)

In [38]:
df_

,Subject,Trial,Saccades #,X_Start,Y_Start,X_End,Y_End
0,18.0,1.0,1.0,936.6,449.8,951.4,450.4
1,18.0,1.0,2.0,952.3,446.7,939.1,446.9
2,18.0,1.0,3.0,940.6,452.8,1008.2,455.8
3,18.0,1.0,4.0,1012.9,456.2,937.6,442.4
4,18.0,1.0,5.0,941.7,445.0,930.2,463.0
...,...,...,...,...,...,...,...
137392,55.0,160.0,9.0,931.4,537.7,963.5,539.7
137393,55.0,160.0,10.0,954.7,566.4,989.5,536.8
137394,55.0,160.0,11.0,990.8,547.5,992.2,510.2
137395,55.0,160.0,12.0,992.4,526.3,1006.5,500.3


In [39]:
df_.to_csv('data/gaze/tmp/tmp_saccades.csv',index=False)

# saccades baseline correction

In [45]:
saccades_updated = df_.copy(deep=True)

In [46]:
# for all subjects
for path in all_roi:
    subject = extract_sub(path)
    baseline = pd.read_csv('./data/gaze/corrected_baselines/'+str(subject)+'.csv')

    for i in range(len(baseline)):
        trial = baseline.TRIAL_LABEL[i]
        valid = baseline.IS_VALID[i]

        if valid:
            x_base = baseline.CURRENT_FIX_X[i]
            y_base = baseline.CURRENT_FIX_Y[i]

        else:
            x_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_X'].mean()
            y_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_Y'].mean()

        delta_x, delta_y = (x_cross-x_base), (y_cross-y_base)

        for row in df_[df_['Trial'] == trial].iterrows():
            saccades_updated.loc[row[0], 'X_Start'] = np.round(row[1]['X_Start'] + delta_x,2)
            saccades_updated.loc[row[0], 'Y_Start'] = np.round(row[1]['Y_Start'] + delta_y, 2)
            saccades_updated.loc[row[0], 'X_End'] = np.round(row[1]['X_End'] + delta_x,2)
            saccades_updated.loc[row[0], 'Y_End'] = np.round(row[1]['Y_End'] + delta_y, 2)

In [47]:
saccades_updated

,Subject,Trial,Saccades #,X_Start,Y_Start,X_End,Y_End
0,18.0,1.0,1.0,986.3,433.7,1001.1,434.3
1,18.0,1.0,2.0,1002.0,430.6,988.8,430.8
2,18.0,1.0,3.0,990.3,436.7,1057.9,439.7
3,18.0,1.0,4.0,1062.6,440.1,987.3,426.3
4,18.0,1.0,5.0,991.4,428.9,979.9,446.9
...,...,...,...,...,...,...,...
137392,55.0,160.0,9.0,986.7,509.2,1018.8,511.2
137393,55.0,160.0,10.0,1010.0,537.9,1044.8,508.3
137394,55.0,160.0,11.0,1046.1,519.0,1047.5,481.7
137395,55.0,160.0,12.0,1047.7,497.8,1061.8,471.8


In [50]:
929.468402777778 - x_cross

-30.53159722222199

In [49]:
saccades_updated.to_csv('data/gaze/saccades.csv',index=False)

# saccades with ROI

In [59]:
saccades_updated['ROI_Start'] = np.nan
saccades_updated['ROI_End'] = np.nan

In [60]:
for i,row in saccades_updated.iterrows():
    x_start = int(row['X_Start'])
    y_start = int(row['Y_Start'])

    x_end = int(row['X_End'])
    y_end = int(row['Y_End'])

    if (x_start,y_start) in eye_roi:
        saccades_updated.loc[i, "ROI_Start"] = "eye"
    if (x_start,y_start) in mouth_nose_roi:
        saccades_updated.loc[i, "ROI_Start"] = "mouth_nose"


    if (x_end,y_end) in eye_roi:
        saccades_updated.loc[i, "ROI_End"] = "eye"
    if (x_end,y_end) in mouth_nose_roi:
        saccades_updated.loc[i, "ROI_End"] = "mouth_nose"


In [61]:
saccades_updated.to_csv('data/gaze/saccades_with_ROI.csv',index=False)

# features ou